In [1]:
# !wget https://f000.backblazeb2.com/file/malaya-model/pretrained/gpt2-345m.tar.gz
# !tar -zxf gpt2-345m.tar.gz
# !rm gpt2-345m.tar.gz
# !wget https://huggingface.co/huseinzol05/bpe/resolve/main/gpt2-encoder.json
# !wget https://huggingface.co/huseinzol05/bpe/resolve/main/gpt2-vocab.bpe

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
import json

x = {
    "n_vocab": 50257,
    "n_ctx": 1024,
    "n_embd": 1024,
    "n_head": 16,
    "n_layer": 24
}
with open('config.json', 'w') as fopen:
    json.dump(x, fopen)

In [4]:
!ls gpt2-345m

checkpoint	 frozen_model.pb.quantized	 model.ckpt.index
frozen_model.pb  model.ckpt.data-00000-of-00001  model.ckpt.meta


In [5]:
!mkdir gpt2-345m-pt

In [6]:
!transformers-cli convert --model_type gpt2 \
  --tf_checkpoint gpt2-345m/model.ckpt \
  --pytorch_dump_output gpt2-345m-pt \
  --config config.json

Converting TensorFlow checkpoint from /home/husein/malaya/gpt2-345m/model.ckpt
Loading TF weight model/h0/attn/c_attn/b with shape [3072]
Loading TF weight model/h0/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h0/attn/c_proj/b with shape [1024]
Loading TF weight model/h0/attn/c_proj/w with shape [1, 1024, 1024]
Loading TF weight model/h0/ln_1/b with shape [1024]
Loading TF weight model/h0/ln_1/g with shape [1024]
Loading TF weight model/h0/ln_2/b with shape [1024]
Loading TF weight model/h0/ln_2/g with shape [1024]
Loading TF weight model/h0/mlp/c_fc/b with shape [4096]
Loading TF weight model/h0/mlp/c_fc/w with shape [1, 1024, 4096]
Loading TF weight model/h0/mlp/c_proj/b with shape [1024]
Loading TF weight model/h0/mlp/c_proj/w with shape [1, 4096, 1024]
Loading TF weight model/h1/attn/c_attn/b with shape [3072]
Loading TF weight model/h1/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h1/attn/c_proj/b with shape [1024]
Loading TF weight model/h1/

In [7]:
from transformers import GPT2LMHeadModel, GPT2Config, GPT2Tokenizer

In [8]:
model = GPT2LMHeadModel.from_pretrained('./gpt2-345m-pt')

In [9]:
tokenizer = GPT2Tokenizer('gpt2-encoder.json', 'gpt2-vocab.bpe')
tokenizer.save_pretrained('gpt2-345m-pt')

('gpt2-345m-pt/tokenizer_config.json',
 'gpt2-345m-pt/special_tokens_map.json',
 'gpt2-345m-pt/vocab.json',
 'gpt2-345m-pt/merges.txt',
 'gpt2-345m-pt/added_tokens.json')

In [10]:
tokenizer.encode('najib razak adalah seorang', return_tensors='pt')

tensor([[  77, 1228,  571,  374, 1031,  461,  512,  282,  993,  384,  273,  648]])

In [11]:
g = model.generate(tokenizer.encode('Najib Razak adalah seorang', return_tensors='pt'),
                  do_sample=True, 
    max_length=100, 
    top_p=0.95, 
    top_k=50)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [12]:
tokenizer.decode(g[0])

"Najib Razak adalah seorang ahli perniagaan yang kaya dan menguntungkan.\nApabila kesunyian menjadi nyata, saya membaca mengenai Perdana Menteri Tony Blair: 'Ada perang selanjutnya?\nSaya terbang ke Washington untuk mesyuarat perniagaan tiga tahun di sana.\n"

In [13]:
model.push_to_hub('gpt2-345m-bahasa-cased', organization='mesolitica')

/home/husein/.local/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:38: FutureWarning: Deprecated positional argument(s) used in 'create_repo': pass token='gpt2-345m-bahasa-cased' as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  warnings.warn(
/home/husein/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:102: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.10. Pass `repo_id` instead.
  warnings.warn(
/home/husein/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:681: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(
Cloning https://huggingface.co/mesolitica/gpt2-345m-bahasa-cased into local empty directory.


Upload file pytorch_model.bin:   0%|          | 32.0k/1.35G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/mesolitica/gpt2-345m-bahasa-cased
   16b6b3c..1c24f8b  main -> main



'https://huggingface.co/mesolitica/gpt2-345m-bahasa-cased/commit/1c24f8bfe0f8541df5794994e32abc5a2b66f3b8'

In [14]:
tokenizer.push_to_hub('gpt2-345m-bahasa-cased', organization='mesolitica')

To https://huggingface.co/mesolitica/gpt2-345m-bahasa-cased
   1c24f8b..ba649e0  main -> main



'https://huggingface.co/mesolitica/gpt2-345m-bahasa-cased/commit/ba649e05582238c96273db4ccb6cd04c535a7c3c'